In [ ]:
from place_order import *

## 下买单后，如果（任何一方）成功立马下best ask卖单，（判断买单是否超时）

In [ ]:
def process_side(client, 
                 token, 
                 current_buy_orders, 
                 current_sell_orders, 
                 market_ids, 
                 token_ids,
                 token_id_side, 
                 size, 
                 question, 
                 buy_order_times,
                #  sell_order_times,
                 best_bid,
                #  best_ask
                 ):
    
    """
    处理单边的订单逻辑：
      1. 检查买单状态；
      2. 当买单成交且该边尚未有卖单时，下对应边的卖单；
      3. 检查卖单状态，如已成交则下新的买单。
    """

    market_id = market_ids.get(token)
    if not market_id:
        logger.error(f"[{question}] 未知的 {token.upper()} 市场信息，无法处理订单。")
    
    ##############################################################################################################################################################################
    ###############################
    # 1. 检查买单状态
    ###############################
    if current_buy_orders.get(token):
        
        buy_order_id = current_buy_orders[token][0].get("id") # 特定问题的 买单的 特定token的 特定id
        best_bid_price = max(float(order.price) for order in client.get_order_book(token_id_side[token]).bids)  # 获取当前的best_bid

        # 检查是否超时
        current_buy_time = int(time.time())
        buy_order_time = buy_order_times[token].get(buy_order_id)  # 获取对应侧的买单创建时间

        ##########################################################################################################################################################
        ####################################################################
        # 1.1 检查买单状态，如果超时并且best bid发生变化那么就cancel重新下单，如果没超时就查看买单是否成交
        ####################################################################
        if current_buy_time - buy_order_time > 9 and float(best_bid_price) != float(best_bid[token]):  # 超过一定时间 并且 最优买单价格发生了变化
            
            logger.warning(f"[{question}] {token.upper()} 买单超时，且最优买单价格发生了变化，取消订单并重新下单。")
            
            # 再取消订单之前要把没完全买进来的shares卖掉
            # updated_orders = client.get_orders(OpenOrderParams(market=market_id))
            logger.info(f"再取消买单之前，判段是否需要把没完全买进来的shares卖掉")
            
            buy_filled_size = float(order.get('size_matched') 
                                for order in client.get_orders(OpenOrderParams(market=market_id))
                                if order.get('id') == buy_order_id
                               )
            # print(buy_filled_size)

            if float(buy_filled_size) != float(0):
                
                logger.info(f'[{question}] 的上一买单 {token.upper()} 有{buy_filled_size}成交')
                place_sell_orders(client, 
                                 token_ids, 
                                 market_id, 
                                 buy_filled_size, 
                                 question, 
                                 token)
                # 因为这里面卖的数量不是一开始定义的完整的size，所以不记录
                
            else:
                logger.error(f"[{question}] 的上一单 {token.upper()} 没有任何买单成交，可以直接cancel")   
            
            
            client.cancel(order_id=buy_order_id) # 取消订单
            
            #####清空######
            current_buy_orders[token] = [] # 1 清空 buy order
            best_bid[token] = [] # 2 清空 best bid
            del buy_order_times[token][buy_order_id] # 3 删除旧的买单时间记录
            
            ##############################################################################################
            # 重新下新的买单
            buy_orders_resp = place_buy_orders(client, 
                                                token_ids, 
                                                market_id, 
                                                size, 
                                                question, 
                                                token)
                
            if buy_orders_resp and buy_orders_resp.get(token):
                logger.info(f"[{question}] {token.upper()} 新买单已下单")
                    
                #####更新######
                current_buy_orders[token] = buy_orders_resp[token] # 1 更新 current_buy_orders
                buy_order_id = current_buy_orders[token][0].get("id") # 2 更新 buy_order_id
                buy_order_times[token][buy_order_id] = current_buy_orders[token][0].get('created_at')  # 3 更新 buy_order_times
                best_bid[token] = current_buy_orders[token][0].get("price") # 4 更新 best bid价格

                
            else:
                logger.error(f"[{question}] {token.upper()} 新买单下单失败。")
            ##############################################################################################
        else:
            logger.error(f"[{question}] {token.upper()} 买单没有超时 或 最优买单价格没有发生变化。") 
        ##########################################################################################################################################################    
    else:
        logger.error(f"[{question}] {token.upper()} 当前无买单挂单。")

    ######################################################################################################################################################################
    
    ##################################
    # 2. 买单全部成交后下卖单（如果该边尚无卖单）
    ##################################
    
    if not any(order.get("id") == buy_order_id for order in client.get_orders(OpenOrderParams(market=market_id))): # 若在更新的订单中找不到该买单，认为订单已全部成交
        logger.info(f"[{question}] {token.upper()} 买单已全部成交。")
                
        #####清空######
        current_buy_orders[token] = [] # 1 清空 buy order
        best_bid[token] = [] # 2 清空 best bid
        del buy_order_times[token][buy_order_id]  # 3 删除成交的买单时间记录
                
    else:
        logger.error(f"[{question}] {token.upper()} 买单仍在挂单中。")

    #########################################################################################
    
    # if not current_buy_orders.get(token) and not current_sell_orders.get(token):
    if not current_sell_orders.get(token): # 如果卖单还没有记录
        logger.info(f"[{question}] {token.upper()} 买单已成交，开始下卖单。")

        sell_orders = place_sell_orders(client, 
                                        token_ids, 
                                        market_id, 
                                        size - 0.01, # 买单全部成交时下的卖单
                                        question, 
                                        token)
            
        if sell_orders and sell_orders.get(token):
                
            #####更新######
            current_sell_orders[token] = sell_orders[token] # 1 更新sell orders
            sell_order_id = current_sell_orders[token][0].get("id") # 2 更新 sell_order_id
            # sell_order_times[token][sell_order_id] = current_sell_orders[token][0].get('created_at') # 3 更新当前 sell_order_times
            # best_ask[token] = current_sell_orders[token][0].get("price") # 4 更新 best_ask 价格


            logger.info(f"[{question}] {token.upper()} 卖单已下单，订单ID: {sell_order_id}")
        else:
            logger.error(f"[{question}] {token.upper()} 卖单下单失败。")
    else:
        logger.error(f"[{question}] {token.upper()} 下卖单条件不成立。")
    #########################################################################################
    time.sleep(2)

    ###########################################################################################################################################################
    
    ###############################
    # 3. 检查卖单状态
    ###############################
    if current_sell_orders.get(token):
            
        # sell_order_id = current_sell_orders[token][0].get("id")
        # best_ask_price = min(float(order.price) for order in client.get_order_book(token_id_side[token]).asks) # 获取best ask

        # # 检查是否超时
        # current_sell_time = int(time.time())
        # sell_order_time = sell_order_times[token].get(sell_order_id)  # 获取对应侧的卖单创建时间


        # ##########################################################################################################################################################
        # ####################################################################
        # # 3.1 检查卖单状态，如果超时并且best ask发生变化那么就cancel重新下单，如果没超时就查看卖单是否成交
        # ####################################################################
        # if current_sell_time - sell_order_time > 9 and float(best_ask_price) != float(best_ask[token]):  # 超过一定时间 并且 最优买单价格发生了变化
            
        #     logger.warning(f"[{question}] {token.upper()} 卖单超时，且最优买单价格发生了变化，取消订单并重新下单。")
            
        #     # # 再取消订单之前要把没完全卖掉的shares卖掉
        #     # logger.info(f"再取消卖单之前，判段是否需要把没完全卖掉的shares卖掉")

        #     # sell_origin_size = float(order.get('original_size') 
        #     #                     for order in client.get_orders(OpenOrderParams(market=market_id))
        #     #                     if order.get('id') == sell_order_id
        #     #                    )

        #     # sell_filled_size = float(order.get('size_matched') 
        #     #                     for order in client.get_orders(OpenOrderParams(market=market_id))
        #     #                     if order.get('id') == sell_order_id
        #     #                    )
        #     # # print(buy_filled_size)

        #     # if float(sell_origin_size - sell_filled_size) > float(0):
                
        #     #     logger.info(f'[{question}] 的上一卖单 {token.upper()} 有{sell_filled_size}成交')
        #     #     place_sell_orders(client, 
        #     #                      token_ids, 
        #     #                      market_id, 
        #     #                      sell_origin_size - sell_filled_size, # 把剩下的单子卖完
        #     #                      question, 
        #     #                      token)
                
        #     # else:
        #     #     logger.error(f"[{question}] 的上一单 {token.upper()} 没有任何卖单成交，可以直接cancel")
            
            
        #     # client.cancel(order_id=sell_order_id)  # 取消订单
        #     # #####清空######
        #     # current_sell_orders[token] = [] # 1 清空 buy order
        #     # best_ask[token] = [] # 2 清空 best bid
        #     # del sell_order_times[token][sell_order_id] # 3 删除旧的买单时间记录
            
        #     ##############################################################################################
        #     # 重新下新的卖单
        #     sell_orders_resp = place_sell_orders(client, 
        #                                         token_ids, 
        #                                         market_id, 
        #                                         size - 0.01, # 下完整的卖单
        #                                         question, 
        #                                         token)
                
        #     if sell_orders_resp and sell_orders_resp.get(token):
        #         logger.info(f"[{question}] {token.upper()} 新卖单已下单")
                    
        #         #####更新######
        #         current_sell_orders[token] = sell_orders_resp[token] # 1 更新 current_sell_orders
        #         sell_order_id = current_sell_orders[token][0].get("id")
        #         sell_order_times[token][sell_order_id] = current_sell_orders[token][0].get('created_at')  # 2 更新 sell_order_times
        #         best_ask[token] = current_sell_orders[token][0].get("price") # 3 更新 best ask价格
                
        #     else:
        #         logger.error(f"[{question}] {token.upper()} 新卖单下单失败。")
        #     ##############################################################################################
        # else:
        #     logger.error(f"[{question}] {token.upper()} 卖单没有超时 或 最优卖单价格没有发生变化。") 
        # ##########################################################################################################################################################    
        
        #####################
        # 3.2 如果所有卖单都成交
        #####################
        if not any(order.get("id") == current_sell_orders[token][0].get("id") for order in client.get_orders(OpenOrderParams(market=market_id))):
            logger.info(f"[{question}] {token.upper()} 卖单已全部成交。")
                
            current_sell_orders[token] = []
                
            ##############################################################################################
            # 重新下新的买单
            buy_orders_resp = place_buy_orders(client, 
                                                token_ids, 
                                                market_id, 
                                                size, 
                                                question, 
                                                token)
                        
            if buy_orders_resp and buy_orders_resp.get(token):
                    
                #####更新######
                current_buy_orders[token] = buy_orders_resp[token] # 1 更新 current_buy_orders
                buy_order_id = current_buy_orders[token][0].get("id")
                buy_order_times[token][buy_order_id] = current_buy_orders[token][0].get('created_at')  # 2 更新 buy_order_times
                best_bid[token] = current_buy_orders[token][0].get("price") # 3 更新 best bid价格

                logger.info(f"[{question}] {token.upper()} 新买单已下单")
            else:
                logger.error(f"[{question}] {token.upper()} 新买单下单失败。")
            ##############################################################################################
                    
        else:
            logger.error(f"[{question}] {token.upper()} 卖单仍在挂单中。")
    else:
        logger.error(f"[{question}] {token.upper()} 当前无卖单挂单。")
    ##############################################################################################################################################################
    time.sleep(2)


In [ ]:

def continuous_trade_loop(client, 
                          market_ids,
                          token_ids, 
                          token_id_side,
                          size, 
                          question, 
                          initial_buy_orders,
                          current_sell_orders, 
                          buy_order_times,
                        #   sell_order_times,
                          best_bid,
                        #   best_ask
                          ):
    
    """
    持续执行买卖循环，逻辑如下：
      1. 初始状态下，已下的买单保存在 buy_orders 中（例如 {'yes': [buy_order_dict], 'no': [buy_order_dict]}）。
      2. 轮询检查每一边的买单状态（调用 client.get_orders(OpenOrderParams(market=market_id)) 判断订单是否存在）。
         如果某边的买单已成交（订单不存在），则立即调用 place_sell_orders 下对应的卖单。
      3. 对于已下的卖单，同样轮询检查卖单状态，如果卖单成交，则调用 place_buy_orders 下新买单，
         使得整个买卖循环持续运行。
      4. 如果一边成交而另一边未成交，则该边继续等待，不影响另一边的交易循环。
      
    每个状态判断后增加 3 秒延迟，给 API 反应时间。

    :param client: 交易客户端对象，需要提供 get_order_book 和 get_orders 方法。
    :param token_ids: 包含 Yes 和 No 的 Token ID 列表（假定 token_ids[0] 对应 'yes'，token_ids[1] 对应 'no'）。
    :param buy_orders: 初始买单字典，例如 {'yes': [buy_order_dict], 'no': [buy_order_dict]}。
    :param size: 买单和卖单的订单数量。
    :param question: 日志标识字符串。
    :param interval: 每轮完整轮询两边后的延迟（秒）。
    """
    
    
    ########################
    ###在一个question里面#####
    ####对于每个yes 和 no#####
    ########################
    
    # current_buy_orders初始化最开始的买单
    current_buy_orders = initial_buy_orders


    while True:
        # 并发处理 'yes' 和 'no' 两边的订单逻辑
        with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
            futures = {
                executor.submit(
                    process_side,
                    client, # 初始化
                    token, # ‘yes’ 和 ‘no’ 两个token
                    current_buy_orders, # 最开始的 卖单记录
                    current_sell_orders, # 最开始的买单记录
                    market_ids, # 每个question的id
                    token_ids, # 列表：记录yes和no的id
                    token_id_side, # 字典： 每个yes和no的 token id映射
                    size, # 下单总share
                    question, # 问题名称
                    buy_order_times, # 买单记录的时间
                    # sell_order_times, # 卖单记录的时间
                    best_bid, # 字典：每个yes和no的best bid映射
                    # best_ask # 字典：每个yes和no的best ask映射
                ): token for token in ['yes', 'no']
            }
            concurrent.futures.wait(futures)



# # buy_orders: {
# #           'no': [{'id': '0x0e3e4d66129d7c1a44ca3e8beb22b7ff5188fdd33351aa7e7c81ec0471046f8e', 
# #                      'status': 'LIVE', 
# #                      'owner': 'cde75286-a1c7-a5b7-b00f-f21ac99f6fba', 
# #                      'maker_address': '0xCA3095B81B1Af8b0096150f065e9c4330d4B8042', 
# #                      'market': '0xaf1a2b4ccf8b92efc3710b5d3bb263aa28c9ecf4858abb6c73047c1c0d7b9416', 
# #                       'asset_id': '52170441229254093239180420557188676391777309638074470699644705957928403317548', 
# #                       'side': 'BUY', 'original_size': '20', 'size_matched': '0', 
# #                       'price': '0.861', 'outcome': 'No', 'expiration': '0', 
# #                       'order_type': 'GTC', 'associate_trades': [], 'created_at': 1739822859}], 

# #             'yes': [{'id': '0xbff46f0f8211d4f4048126a101e8108fbb80fd594e48412c6cfde619389ce745', 
# #                      'status': 'LIVE', 
# #                       'owner': 'cde75286-a1c7-a5b7-b00f-f21ac99f6fba', 
# #                      'maker_address': '0xCA3095B81B1Af8b0096150f065e9c4330d4B8042', 
# #                      'market': '0xaf1a2b4ccf8b92efc3710b5d3bb263aa28c9ecf4858abb6c73047c1c0d7b9416', 
# #                      'asset_id': '50590995350569541543130773217099833464734527698164475480567020627331537312844', 
# #                       'side': 'BUY', 'original_size': '20', 'size_matched': '0', 
# #                       'price': '0.134', 'outcome': 'Yes', 'expiration': '0', 'order_type': 'GTC', 
# #                       'associate_trades': [], 'created_at': 1739822859}]
# #                       }

## 总执行

In [ ]:

########################
########################
########################
def trade_pair(client, trade_dict, size):
    """
    同时处理多个 token 交易对（并行下单），交易字典格式为：
        key: 问题，
        value: 包含 'TokenID'、'conditionId'、'spread' 的字典
    每笔交易时将 conditionId 与 spread 信息也包含在日志中，方便确认每笔订单对应的条件。
    该函数将一直运行，持续进行买卖滚动交易。
    
    :param client: 交易客户端对象
    :param trade_dict: 交易字典，例如：
           {
               "问题1": {"TokenID": [yes_token_id, no_token_id], "conditionId": market_id1, "spread": spread1},
               "问题2": {"TokenID": [yes_token_id, no_token_id], "conditionId": market_id2, "spread": spread2},
               ...
           }
    :param size: 每笔订单的数量
    :return: 该函数不会退出，除非出现异常或被外部终止
    """

    def execute_trade(question, trade_info):
        token_ids = trade_info.get("TokenID")
        conditionId = trade_info.get("conditionId")
        spread = trade_info.get("spread")

        
        ########################
        ####对于每个question#####
        ########################


        # 将问题、条件ID 与 spread 信息组合到日志标识中
        header = f"{question} (market condition: {conditionId}, spread: {spread})"
        logger.info(f"开始交易: {header}")


        # 1
        # 保存每边的 market_id 信息，防止后续订单列表清空后无法获取 market_id

        market_ids = {'yes': conditionId, 'no': conditionId}

        # 2
        # 保存每个yes或者no的 token_id 信息，防止后续订单列表清空后无法获取 token_id
        token_id_side ={'yes': token_ids[0], 'no': token_ids[1]} 

        
        # 3
        # 为每个 question 创建独立的 initial_buy_orders
        initial_buy_orders = {
                                "yes": place_buy_orders(client, token_ids, conditionId, size, question=question, side="yes").get("yes"),
                                "no": place_buy_orders(client, token_ids, conditionId, size, question=question, side="no").get("no")
                                }

        if not all(initial_buy_orders.values()):
                logger.error(f"{question} 初始买单下单失败，终止该交易。")
                return

        logger.info(f"{question} 初始买单下单成功: {initial_buy_orders}")

        
        # # 4
        # # 假设在外层定义一个 unsold_number 字典，用来记录每个 side 累计待卖数量
        # unsold_number = {'yes': initial_buy_orders['yes'][0].get('original_size') - initial_buy_orders['yes'][0].get('size_matched'),
        #              'no': initial_buy_orders['no'][0].get('original_size') - initial_buy_orders['no'][0].get('size_matched')
        #              }

        # 5
        # 为每个 question 创建独立的 buy_order_times
        buy_order_times = {
                            side: {orders[0].get("id"): orders[0].get("created_at")}
                            for side, orders in initial_buy_orders.items() if orders
                          }

    
        # 6
        # 为每个 question 创建独立的 best_bid
        # 记录下单时最佳买价
        best_bid = {
                    'yes': initial_buy_orders['yes'][0].get("price"),
                    'no': initial_buy_orders['no'][0].get("price")
                    }
        
        # 7
        # 为每个 question 创建独立的 current_sell_orders
        # 记录卖单下单时的order
        current_sell_orders = {'yes': [], 'no': []}

        
        # # 8
        # # 为每个 question 创建独立的 sell_order_times
        # sell_order_times = {'yes': [], 'no': []}

        # # 9
        # # 为每个 question 创建独立的 best_ask
        # # 记录下单时最佳卖价
        # best_ask = {'yes': [], 'no': []}
        
        ###################################
        # 启动持续交易循环，持续监控并处理买卖订单
        ###################################
        continuous_trade_loop(client, 
                              market_ids,
                              token_ids, 
                              token_id_side,
                              size, 
                              question, 
                              initial_buy_orders,
                              current_sell_orders, 
                              buy_order_times,
                            #   sell_order_times,
                              best_bid,
                            #   best_ask
                              ) # 这个函数一直在循环
    
    # 对每个交易对并行启动交易线程
    with concurrent.futures.ThreadPoolExecutor(len(trade_dict)) as executor:
        futures = []
        for question, trade_info in trade_dict.items():
            futures.append(executor.submit(execute_trade, 
                                           question, 
                                           trade_info)) # 为每个question 下初始买单，然后开始loop交易
        
        # 持续交易循环不会退出，除非出错或外部终止
        concurrent.futures.wait(futures)

        
        
# initial_buy_orders: {
#           'no': [{'id': '0x0e3e4d66129d7c1a44ca3e8beb22b7ff5188fdd33351aa7e7c81ec0471046f8e', 
#                      'status': 'LIVE', 
#                      'owner': 'cde75286-a1c7-a5b7-b00f-f21ac99f6fba', 
#                      'maker_address': '0xCA3095B81B1Af8b0096150f065e9c4330d4B8042', 
#                      'market': '0xaf1a2b4ccf8b92efc3710b5d3bb263aa28c9ecf4858abb6c73047c1c0d7b9416', 
#                       'asset_id': '52170441229254093239180420557188676391777309638074470699644705957928403317548', 
#                       'side': 'BUY', 'original_size': '20', 'size_matched': '0', 
#                       'price': '0.861', 'outcome': 'No', 'expiration': '0', 
#                       'order_type': 'GTC', 'associate_trades': [], 'created_at': 1739822859}], 

#             'yes': [{'id': '0xbff46f0f8211d4f4048126a101e8108fbb80fd594e48412c6cfde619389ce745', 
#                      'status': 'LIVE', 
#                       'owner': 'cde75286-a1c7-a5b7-b00f-f21ac99f6fba', 
#                      'maker_address': '0xCA3095B81B1Af8b0096150f065e9c4330d4B8042', 
#                      'market': '0xaf1a2b4ccf8b92efc3710b5d3bb263aa28c9ecf4858abb6c73047c1c0d7b9416', 
#                      'asset_id': '50590995350569541543130773217099833464734527698164475480567020627331537312844', 
#                       'side': 'BUY', 'original_size': '20', 'size_matched': '0', 
#                       'price': '0.134', 'outcome': 'Yes', 'expiration': '0', 'order_type': 'GTC', 
#                       'associate_trades': [], 'created_at': 1739822859}]
#                       }
        


## 没完全成交的订单状态
# [{'id': '0x56d2d840a9401ec9c4851426aac8ae6d86b12763a5c3600c2c5e11df037007ca',
#   'status': 'LIVE',
#   'owner': 'da9dceae-36b9-5b25-efed-a720b85a68dc',
#   'maker_address': '0x7aC4Fb15368d2b62C8ebb6B9B50F8b568f0dd649',
#   'market': '0x2090c30d181142250d1f25b5da4808b16cddd7f2e2cfcf19bdfd508325498fb6',
#   'asset_id': '89768568686584036399418990202600408525021143064935199774078963108133813129029',
#   'side': 'SELL',
#   'original_size': '40',
#   'size_matched': '10',
#   'price': '0.61',
#   'outcome': 'Yes',
#   'expiration': '0',
#   'order_type': 'GTC',
#   'associate_trades': ['127414fc-cdcd-4116-9829-33130f308c81'],
#   'created_at': 1740017091}]

In [ ]:

  # 示例：定义包含问题和对应 token_id 列表的字典
trade_tokens_dict = {
        
# 'Dan Clancy out as Twitch CEO in 2025?': {'TokenID': ['52521422646418218239213208751328222694757717504934616636622590621075679371215',
#    '66390163516315956466696118534227657995267232292264828573695652317434995942865'],
#   'conditionId': '0x08a5f56fdeacd0d3adbc618d12f73927d51a5654fd3883fad95ed863209deda9',
#   'spread': 0.12},
#  'U.S. Government funding lapse on March 15?': {'TokenID': ['38236598342491303152364421973300502578732152123493806980881863251221928118496',
#    '110029657107373809870733195431083018188617129571551255913640257442170639926312'],
#   'conditionId': '0xed85b5d16739e076cd5cc3a705467df14abc2f23d1b16ae26cd660e8e0b38c54',
#   'spread': 0.06},
 'Will Alex Ovechkin break the scoring record this season?': {'TokenID': ['63663749132572596873908667896561033995247956576658486486562811443872179105859',
   '42851955315071033911893366631439515039062582184134718446803053180766367743297'],
  'conditionId': '0xf1d4991800e45fb605dface8076828f544f0942d43bb860c7143cc50f5afa0db',
  'spread': 0.14},
 'DeepSeek R2 released before May?': {'TokenID': ['29963162115904832630744999864898726174740942941070452699509602715653591930448',
   '15838501341409637883188614082285793303061195616446522350017583873331861181564'],
  'conditionId': '0xd3dd91c568a792cd21d986bfda8b68199d9f88bad3586a34f8761ce51c71080e',
  'spread': 0.05},
  'Will Evan Mobley win 2024-25 NBA Defensive Player of the Year?': {'TokenID': ['105162188562847400289458518761958778774034547808595994300188054650302510302762',
   '6039792870513708491203598533719103208431906026446537078045410319950012559121'],
  'conditionId': '0xfe9bdb0f1076b96248f6d1700e569a9ae97512a815c8d913a9617f937c14c353',
  'spread': 0.091},
 'Will Andrew Tate leave Romania before April?': {'TokenID': ['24051239369425675499038252794593635888350152008730181365561196286690381206467',
   '36965501175159678572732441088422857549182456365442012420771263568972010077930'],
  'conditionId': '0x9953104942de71b93a962162dfd5edc3fe673d26e1a4131d12d7ee5423fdf476',
  'spread': 0.36}
  }

# 运行交易
trade_pair(client, trade_tokens_dict, size=10)


In [ ]:
# resp = client.cancel(order_id="0xfffe5fd2d6ebb4a7b15bc809bd2bc9214073c277d8a3fcfdf77f2a57f9afe29d")

In [ ]:
# resp = client.get_orders(
#         OpenOrderParams(
#             market="0xb1e4c60e9cb784c7f6fe06eb5572346643cac9e48645c2fa72ff9fab6e38ec74",
#         )
#     )

# resp
# client.get_order_book('52521422646418218239213208751328222694757717504934616636622590621075679371215').bids